In [7]:
#Import dependencies
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score
import pandas as pd
import numpy as np
import tensorflow as tf

# Import findspark and initialize. 
import findspark
findspark.init()

# Start Spark session
import pyspark
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("Asteroid DataFrame").getOrCreate()
spark.conf.set("spark.sql.execution.arrow.pyspark.enabled", "true")

In [8]:
# Read in data from S3 Buckets
from pyspark import SparkFiles
url = "https://s3.amazonaws.com/kat-project4-csv-site.click/dataset.csv"
spark.sparkContext.addFile(url)
spark_df = spark.read.csv(SparkFiles.get("dataset.csv"), sep=",", header=True)

# Show DataFrame
spark_df.show(10)


#If this takes forever to load. Interupt the kernal and then run it and it should load quickly. 

+--------+-------+--------------+----+-------+------+---+---+----+--------+------+--------------+--------+---------+---------+----------------+-------+------------------+-----------------+-----------------+-----------------+-----------------+-----------------+-----------------+-----------------+-----------------+--------------------+----------------+-----------------+----------------+-------+------------+----------+----------+----------+---------+---------+---------+---------+----------+----------+---------+---------+-----+------+
|      id|  spkid|     full_name|pdes|   name|prefix|neo|pha|   H|diameter|albedo|diameter_sigma|orbit_id|    epoch|epoch_mjd|       epoch_cal|equinox|                 e|                a|                q|                i|               om|                w|               ma|               ad|                n|                  tp|          tp_cal|              per|           per_y|   moid|     moid_ld|   sigma_e|   sigma_a|   sigma_q|  sigma_i| sigma_om|  

In [9]:
#Convert the spark df to pandas
df = spark_df.select("*").toPandas()

In [10]:
#display the pandas DF
df.head()

,id,spkid,full_name,pdes,name,prefix,neo,pha,H,diameter,...,sigma_i,sigma_om,sigma_w,sigma_ma,sigma_ad,sigma_n,sigma_tp,sigma_per,class,rms
0,a0000001,2000001,1 Ceres,1,Ceres,None,N,N,3.4,939.4,...,4.6089E-9,6.1688E-8,6.6248E-8,7.8207E-9,1.1113E-11,1.1965E-12,3.7829E-8,9.4159E-9,MBA,.43301
1,a0000002,2000002,2 Pallas,2,Pallas,None,N,N,4.2,545,...,3.4694E-6,6.2724E-6,9.1282E-6,8.8591E-6,4.9613E-9,4.6536E-10,4.0787E-5,3.6807E-6,MBA,.35936
2,a0000003,2000003,3 Juno,3,Juno,None,N,N,5.33,246.596,...,3.2231E-6,1.6646E-5,1.7721E-5,8.1104E-6,4.3639E-9,4.4134E-10,3.5288E-5,3.1072E-6,MBA,.33848
3,a0000004,2000004,4 Vesta,4,Vesta,None,N,N,3.0,525.4,...,2.1706E-7,3.8808E-7,1.7893E-7,1.2068E-6,1.6486E-9,2.6125E-10,4.1037E-6,1.2749E-6,MBA,.3998
4,a0000005,2000005,5 Astraea,5,Astraea,None,N,N,6.9,106.699,...,2.7408E-6,2.8949E-5,2.9842E-5,8.3038E-6,4.729E-9,5.5227E-10,3.4743E-5,3.4905E-6,MBA,.52191


In [11]:
#review all the column names
df.columns

Index(['id', 'spkid', 'full_name', 'pdes', 'name', 'prefix', 'neo', 'pha', 'H',
       'diameter', 'albedo', 'diameter_sigma', 'orbit_id', 'epoch',
       'epoch_mjd', 'epoch_cal', 'equinox', 'e', 'a', 'q', 'i', 'om', 'w',
       'ma', 'ad', 'n', 'tp', 'tp_cal', 'per', 'per_y', 'moid', 'moid_ld',
       'sigma_e', 'sigma_a', 'sigma_q', 'sigma_i', 'sigma_om', 'sigma_w',
       'sigma_ma', 'sigma_ad', 'sigma_n', 'sigma_tp', 'sigma_per', 'class',
       'rms'],
      dtype='object')

In [12]:
#Drop unnecessary columns that don't determine the hazardousness of an asteroid
df= df.drop(columns=['spkid','pdes','full_name','name','prefix','orbit_id','equinox'])
df.head()

,id,neo,pha,H,diameter,albedo,diameter_sigma,epoch,epoch_mjd,epoch_cal,...,sigma_i,sigma_om,sigma_w,sigma_ma,sigma_ad,sigma_n,sigma_tp,sigma_per,class,rms
0,a0000001,N,N,3.4,939.4,0.090,0.2,2458600.5,58600,20190427.0000000,...,4.6089E-9,6.1688E-8,6.6248E-8,7.8207E-9,1.1113E-11,1.1965E-12,3.7829E-8,9.4159E-9,MBA,.43301
1,a0000002,N,N,4.2,545,0.101,18,2459000.5,59000,20200531.0000000,...,3.4694E-6,6.2724E-6,9.1282E-6,8.8591E-6,4.9613E-9,4.6536E-10,4.0787E-5,3.6807E-6,MBA,.35936
2,a0000003,N,N,5.33,246.596,0.214,10.594,2459000.5,59000,20200531.0000000,...,3.2231E-6,1.6646E-5,1.7721E-5,8.1104E-6,4.3639E-9,4.4134E-10,3.5288E-5,3.1072E-6,MBA,.33848
3,a0000004,N,N,3.0,525.4,0.4228,0.2,2458600.5,58600,20190427.0000000,...,2.1706E-7,3.8808E-7,1.7893E-7,1.2068E-6,1.6486E-9,2.6125E-10,4.1037E-6,1.2749E-6,MBA,.3998
4,a0000005,N,N,6.9,106.699,0.274,3.140,2459000.5,59000,20200531.0000000,...,2.7408E-6,2.8949E-5,2.9842E-5,8.3038E-6,4.729E-9,5.5227E-10,3.4743E-5,3.4905E-6,MBA,.52191


In [14]:
#check number of entries in the dataset
len(df.index)

958524

In [37]:
#drop any rows with null values in them
df= df.dropna()


In [38]:
#check the number of entries after dropping rows
len(df.index)

131142

In [39]:
#set first column as the index
df = df.set_index('id')

In [40]:
#review datatypes
df.dtypes

neo                object
pha                object
H                 float64
diameter          float64
albedo            float64
diameter_sigma    float64
epoch             float64
epoch_mjd           int64
epoch_cal         float64
e                 float64
a                 float64
q                 float64
i                 float64
om                float64
w                 float64
ma                float64
ad                float64
n                 float64
tp                float64
tp_cal            float64
per               float64
per_y             float64
moid              float64
moid_ld           float64
sigma_e           float64
sigma_a           float64
sigma_q           float64
sigma_i           float64
sigma_om          float64
sigma_w           float64
sigma_ma          float64
sigma_ad          float64
sigma_n           float64
sigma_tp          float64
sigma_per         float64
class              object
rms               float64
dtype: object

In [41]:
#Encode neo and pha N and Y values as 0s and 1s

# Create a function to encode the N and Y values
def encode(value):
    if value =="Y":
        return 1
    else:
        return 0

# Call the encode_function on the neo and pha columns
df["neo"] = df["neo"].apply(encode)
df["pha"] = df["pha"].apply(encode)

#Review the df
df.head()

,neo,pha,H,diameter,albedo,diameter_sigma,epoch,epoch_mjd,epoch_cal,e,...,sigma_i,sigma_om,sigma_w,sigma_ma,sigma_ad,sigma_n,sigma_tp,sigma_per,class,rms
id,,,,,,,,,,,,,,,,,,,,,
a0000001,0,0,3.40,939.400,0.0900,0.200,2458600.5,58600,20190427.0,0.076009,...,4.608900e-09,6.168800e-08,6.624800e-08,7.820700e-09,1.111300e-11,1.196500e-12,3.782900e-08,9.415900e-09,MBA,0.43301
a0000002,0,0,4.20,545.000,0.1010,18.000,2459000.5,59000,20200531.0,0.229972,...,3.469400e-06,6.272400e-06,9.128200e-06,8.859100e-06,4.961300e-09,4.653600e-10,4.078700e-05,3.680700e-06,MBA,0.35936
a0000003,0,0,5.33,246.596,0.2140,10.594,2459000.5,59000,20200531.0,0.256936,...,3.223100e-06,1.664600e-05,1.772100e-05,8.110400e-06,4.363900e-09,4.413400e-10,3.528800e-05,3.107200e-06,MBA,0.33848
a0000004,0,0,3.00,525.400,0.4228,0.200,2458600.5,58600,20190427.0,0.088721,...,2.170600e-07,3.880800e-07,1.789300e-07,1.206800e-06,1.648600e-09,2.612500e-10,4.103700e-06,1.274900e-06,MBA,0.39980
a0000005,0,0,6.90,106.699,0.2740,3.140,2459000.5,59000,20200531.0,0.190913,...,2.740800e-06,2.894900e-05,2.984200e-05,8.303800e-06,4.729000e-09,5.522700e-10,3.474300e-05,3.490500e-06,MBA,0.52191


In [46]:
df['class'].unique()

array(['MBA', 'OMB', 'MCA', 'AMO', 'IMB', 'TJN', 'APO', 'ATE', 'CEN',
       'AST', 'TNO'], dtype=object)

In [47]:
#Encode class values as numbers

# Create a function to encode the N and Y values
def encode2(value):
    if value =="MBA":
        return 0
    elif value =="OMB":
        return 1
    elif value =="MCA":
        return 2
    elif value =="AMO":
        return 3
    elif value =="IMB":
        return 4
    elif value =="TJN":
        return 5
    elif value =="APO":
        return 6
    elif value =="ATE":
        return 7
    elif value =="CEN":
        return 8
    elif value =="AST":
        return 9
    else:
        return 10

# Call the encode_function on the neo and pha columns
df["class"] = df["class"].apply(encode2)

#Review the df
df.head()

,neo,pha,H,diameter,albedo,diameter_sigma,epoch,epoch_mjd,epoch_cal,e,...,sigma_i,sigma_om,sigma_w,sigma_ma,sigma_ad,sigma_n,sigma_tp,sigma_per,class,rms
id,,,,,,,,,,,,,,,,,,,,,
a0000001,0,0,3.40,939.400,0.0900,0.200,2458600.5,58600,20190427.0,0.076009,...,4.608900e-09,6.168800e-08,6.624800e-08,7.820700e-09,1.111300e-11,1.196500e-12,3.782900e-08,9.415900e-09,0,0.43301
a0000002,0,0,4.20,545.000,0.1010,18.000,2459000.5,59000,20200531.0,0.229972,...,3.469400e-06,6.272400e-06,9.128200e-06,8.859100e-06,4.961300e-09,4.653600e-10,4.078700e-05,3.680700e-06,0,0.35936
a0000003,0,0,5.33,246.596,0.2140,10.594,2459000.5,59000,20200531.0,0.256936,...,3.223100e-06,1.664600e-05,1.772100e-05,8.110400e-06,4.363900e-09,4.413400e-10,3.528800e-05,3.107200e-06,0,0.33848
a0000004,0,0,3.00,525.400,0.4228,0.200,2458600.5,58600,20190427.0,0.088721,...,2.170600e-07,3.880800e-07,1.789300e-07,1.206800e-06,1.648600e-09,2.612500e-10,4.103700e-06,1.274900e-06,0,0.39980
a0000005,0,0,6.90,106.699,0.2740,3.140,2459000.5,59000,20200531.0,0.190913,...,2.740800e-06,2.894900e-05,2.984200e-05,8.303800e-06,4.729000e-09,5.522700e-10,3.474300e-05,3.490500e-06,0,0.52191


In [48]:
# Remove target (pha) from features data
y = df.pha.values
X = df.drop(columns="pha").values


# Split training/test datasets
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42, stratify=y)

In [49]:
# Preprocess numerical data for neural network

# Create a StandardScaler instances
scaler = StandardScaler()

# Fit the StandardScaler
X_scaler = scaler.fit(X_train)

# Scale the data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

First Model

In [51]:
# Define the deep learning model 
nn_model = tf.keras.models.Sequential()
nn_model.add(tf.keras.layers.Dense(units=36, activation="relu", input_dim=36))
nn_model.add(tf.keras.layers.Dense(units=16, activation="relu"))
nn_model.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))

# Compile the Sequential model together and customize metrics
nn_model.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

# Train the model
fit_model = nn_model.fit(X_train, y_train, epochs=10)

# Evaluate the model using the test data
model_loss, model_accuracy = nn_model.evaluate(X_test,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

Epoch 1/10
3074/3074 [==============================] - 32s 9ms/step - loss: 3867.5671 - accuracy: 0.9950
Epoch 2/10
3074/3074 [==============================] - 31s 10ms/step - loss: 2761.2664 - accuracy: 0.9972
Epoch 3/10
3074/3074 [==============================] - 31s 10ms/step - loss: 2430.2292 - accuracy: 0.9968
Epoch 4/10
3074/3074 [==============================] - 31s 10ms/step - loss: 1727.9470 - accuracy: 0.9970
Epoch 5/10
3074/3074 [==============================] - 27s 9ms/step - loss: 986.0316 - accuracy: 0.9971
Epoch 6/10
3074/3074 [==============================] - 26s 8ms/step - loss: 547.6282 - accuracy: 0.9963
Epoch 7/10
3074/3074 [==============================] - 26s 8ms/step - loss: 209.2773 - accuracy: 0.9967
Epoch 8/10
3074/3074 [==============================] - 27s 9ms/step - loss: 106.9082 - accuracy: 0.9973
Epoch 9/10
3074/3074 [==============================] - 26s 8ms/step - loss: 8.0716 - accuracy: 0.9979
Epoch 10/10
3074/3074 [===========================

Notes: Our first model was very accurate (99.86% accuracy). Future models will see if we can achieve the same accuaracy, with less data. 

Optimization Attempt #1

In [52]:
#Copy the processed df
df_no_sigma = df.copy()

#remove all "simgma" columns from the DF
df_no_sigma = df_no_sigma.drop(columns=['diameter_sigma','sigma_e', 'sigma_a', 'sigma_q', 'sigma_i', 'sigma_om', 'sigma_w','sigma_ma', 'sigma_ad', 'sigma_n', 'sigma_tp', 'sigma_per'])
df_no_sigma.head()

,neo,pha,H,diameter,albedo,epoch,epoch_mjd,epoch_cal,e,a,...,ad,n,tp,tp_cal,per,per_y,moid,moid_ld,class,rms
id,,,,,,,,,,,,,,,,,,,,,
a0000001,0,0,3.40,939.400,0.0900,2458600.5,58600,20190427.0,0.076009,2.769165,...,2.979647,0.213885,2.458239e+06,2.018043e+07,1683.145703,4.608202,1.59478,620.640533,0,0.43301
a0000002,0,0,4.20,545.000,0.1010,2459000.5,59000,20200531.0,0.229972,2.773841,...,3.411748,0.213345,2.458321e+06,2.018072e+07,1687.410992,4.619880,1.23429,480.348639,0,0.35936
a0000003,0,0,5.33,246.596,0.2140,2459000.5,59000,20200531.0,0.256936,2.668285,...,3.353865,0.226129,2.458446e+06,2.018112e+07,1592.013769,4.358696,1.03429,402.514639,0,0.33848
a0000004,0,0,3.00,525.400,0.4228,2458600.5,58600,20190427.0,0.088721,2.361418,...,2.570926,0.271609,2.458248e+06,2.018051e+07,1325.432763,3.628837,1.13948,443.451432,0,0.39980
a0000005,0,0,6.90,106.699,0.2740,2459000.5,59000,20200531.0,0.190913,2.574037,...,3.065455,0.238661,2.458926e+06,2.020032e+07,1508.414421,4.129814,1.09575,426.433027,0,0.52191


In [53]:
# Remove target (pha) from features data
y = df_no_sigma.pha.values
X = df_no_sigma.drop(columns="pha").values


# Split training/test datasets
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42, stratify=y)

In [54]:
# Preprocess numerical data for neural network

# Create a StandardScaler instances
scaler = StandardScaler()

# Fit the StandardScaler
X_scaler = scaler.fit(X_train)

# Scale the data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

In [55]:
# Define the deep learning model 
nn_model = tf.keras.models.Sequential()
nn_model.add(tf.keras.layers.Dense(units=24, activation="relu", input_dim=24))
nn_model.add(tf.keras.layers.Dense(units=12, activation="relu"))
nn_model.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))

# Compile the Sequential model together and customize metrics
nn_model.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

# Train the model
fit_model = nn_model.fit(X_train, y_train, epochs=10)

# Evaluate the model using the test data
model_loss, model_accuracy = nn_model.evaluate(X_test,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

Epoch 1/10
3074/3074 [==============================] - 18s 5ms/step - loss: 24425.8105 - accuracy: 0.9899
Epoch 2/10
3074/3074 [==============================] - 19s 6ms/step - loss: 63.4665 - accuracy: 0.9976
Epoch 3/10
3074/3074 [==============================] - 15s 5ms/step - loss: 1.0628 - accuracy: 0.9948
Epoch 4/10
3074/3074 [==============================] - 15s 5ms/step - loss: 0.0495 - accuracy: 0.9986
Epoch 5/10
3074/3074 [==============================] - 19s 6ms/step - loss: 0.0182 - accuracy: 0.9986
Epoch 6/10
3074/3074 [==============================] - 16s 5ms/step - loss: 0.0118 - accuracy: 0.9986
Epoch 7/10
3074/3074 [==============================] - 15s 5ms/step - loss: 0.0107 - accuracy: 0.9986
Epoch 8/10
3074/3074 [==============================] - 19s 6ms/step - loss: 0.0105 - accuracy: 0.9986
Epoch 9/10
3074/3074 [==============================] - 33s 11ms/step - loss: 0.0105 - accuracy: 0.9986
Epoch 10/10
3074/3074 [==============================] - 29s 9ms/st

Notes: Sigma values in the data set don't impact the accuracy of training in determining if an asteroid is a potential hazard. 

Optimization Attempt #2

In [59]:
#verifying that pha (potentialy hazardous asteroid) and neo (near earth object are not a total 1 to 1 relationship)
print(df['pha'].value_counts())

print(df['neo'].value_counts())

pha
0    130961
1       181
Name: count, dtype: int64
neo
0    130403
1       739
Name: count, dtype: int64


In [61]:
df_no_sigma.columns

Index(['neo', 'pha', 'H', 'diameter', 'albedo', 'epoch', 'epoch_mjd',
       'epoch_cal', 'e', 'a', 'q', 'i', 'om', 'w', 'ma', 'ad', 'n', 'tp',
       'tp_cal', 'per', 'per_y', 'moid', 'moid_ld', 'class', 'rms'],
      dtype='object')

In [62]:
#copy df with sigma coplumns removed
df3 = df_no_sigma.copy()

#remove additional columns of data
df3 = df3.drop(columns=['albedo', 'epoch', "epoch_mjd",	"epoch_cal", "e", "a", "q", "i", "om", "w", "ma", "ad", "n", "tp", "tp_cal",	"per", "per_y", "moid_ld"])

#display the new df
df3.head()


,neo,pha,H,diameter,moid,class,rms
id,,,,,,,
a0000001,0,0,3.40,939.400,1.59478,0,0.43301
a0000002,0,0,4.20,545.000,1.23429,0,0.35936
a0000003,0,0,5.33,246.596,1.03429,0,0.33848
a0000004,0,0,3.00,525.400,1.13948,0,0.39980
a0000005,0,0,6.90,106.699,1.09575,0,0.52191


In [63]:
# Remove target (pha) from features data
y = df3.pha.values
X = df3.drop(columns="pha").values


# Split training/test datasets
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42, stratify=y)

In [66]:
# Preprocess numerical data for neural network

# Create a StandardScaler instances
scaler = StandardScaler()

# Fit the StandardScaler
X_scaler = scaler.fit(X_train)

# Scale the data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

In [67]:
# Define the deep learning model 
nn_model = tf.keras.models.Sequential()
nn_model.add(tf.keras.layers.Dense(units=6, activation="relu", input_dim=6))
nn_model.add(tf.keras.layers.Dense(units=3, activation="relu"))
nn_model.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))

# Compile the Sequential model together and customize metrics
nn_model.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

# Train the model
fit_model = nn_model.fit(X_train, y_train, epochs=10)

# Evaluate the model using the test data
model_loss, model_accuracy = nn_model.evaluate(X_test,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

Epoch 1/10
3074/3074 [==============================] - 26s 7ms/step - loss: 0.1557 - accuracy: 0.9506
Epoch 2/10
3074/3074 [==============================] - 21s 7ms/step - loss: 0.0034 - accuracy: 0.9986
Epoch 3/10
3074/3074 [==============================] - 22s 7ms/step - loss: 0.0027 - accuracy: 0.9986
Epoch 4/10
3074/3074 [==============================] - 21s 7ms/step - loss: 0.0026 - accuracy: 0.9987
Epoch 5/10
3074/3074 [==============================] - 22s 7ms/step - loss: 0.0024 - accuracy: 0.9987
Epoch 6/10
3074/3074 [==============================] - 22s 7ms/step - loss: 0.0023 - accuracy: 0.9988
Epoch 7/10
3074/3074 [==============================] - 22s 7ms/step - loss: 0.0022 - accuracy: 0.9989
Epoch 8/10
3074/3074 [==============================] - 21s 7ms/step - loss: 0.0021 - accuracy: 0.9990
Epoch 9/10
3074/3074 [==============================] - 23s 8ms/step - loss: 0.0021 - accuracy: 0.9990
Epoch 10/10
3074/3074 [==============================] - 22s 7ms/step - l

Less information about the asteroid actually increased the accuracy of the model from 99.86% to 99.90%

Optimization Attempt #3

In [69]:
df4 = df3.copy()

df4= df4.drop(columns=['moid', 'class', 'rms']) 

df4.head()

,neo,pha,H,diameter
id,,,,
a0000001,0,0,3.40,939.400
a0000002,0,0,4.20,545.000
a0000003,0,0,5.33,246.596
a0000004,0,0,3.00,525.400
a0000005,0,0,6.90,106.699


In [70]:
# Remove target (pha) from features data
y = df4.pha.values
X = df4.drop(columns="pha").values


# Split training/test datasets
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42, stratify=y)

In [71]:
# Preprocess numerical data for neural network

# Create a StandardScaler instances
scaler = StandardScaler()

# Fit the StandardScaler
X_scaler = scaler.fit(X_train)

# Scale the data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

In [72]:
# Define the deep learning model 
nn_model = tf.keras.models.Sequential()
nn_model.add(tf.keras.layers.Dense(units=3, activation="relu", input_dim=3))
nn_model.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))

# Compile the Sequential model together and customize metrics
nn_model.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

# Train the model
fit_model = nn_model.fit(X_train, y_train, epochs=10)

# Evaluate the model using the test data
model_loss, model_accuracy = nn_model.evaluate(X_test,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

Epoch 1/10
3074/3074 [==============================] - 23s 7ms/step - loss: 0.0196 - accuracy: 0.9986
Epoch 2/10
3074/3074 [==============================] - 21s 7ms/step - loss: 0.0078 - accuracy: 0.9986
Epoch 3/10
3074/3074 [==============================] - 21s 7ms/step - loss: 0.0051 - accuracy: 0.9986
Epoch 4/10
3074/3074 [==============================] - 21s 7ms/step - loss: 0.0042 - accuracy: 0.9986
Epoch 5/10
3074/3074 [==============================] - 21s 7ms/step - loss: 0.0040 - accuracy: 0.9986
Epoch 6/10
3074/3074 [==============================] - 21s 7ms/step - loss: 0.0038 - accuracy: 0.9986
Epoch 7/10
3074/3074 [==============================] - 22s 7ms/step - loss: 0.0037 - accuracy: 0.9986
Epoch 8/10
3074/3074 [==============================] - 22s 7ms/step - loss: 0.0037 - accuracy: 0.9986
Epoch 9/10
3074/3074 [==============================] - 23s 8ms/step - loss: 0.0036 - accuracy: 0.9986
Epoch 10/10
3074/3074 [==============================] - 22s 7ms/step - l

Accuracy went back down to our starting point. Removing moid, class, and rms did help determine which asteroids are hazardous and which are not.

Final Optimization Attempt

Df3 gave us the most accurate Model. In this final attempt, we will recreate the df only using these columns and removing null values from these columns. This will ideally give us more data to train the model with. 

In [87]:
df5 = pd.read_csv('Resources\dataset.csv', low_memory=False)

df5.head()

,id,spkid,full_name,pdes,name,prefix,neo,pha,H,diameter,...,sigma_i,sigma_om,sigma_w,sigma_ma,sigma_ad,sigma_n,sigma_tp,sigma_per,class,rms
0,a0000001,2000001,1 Ceres,1,Ceres,NaN,N,N,3.40,939.400,...,4.608900e-09,6.168800e-08,6.624800e-08,7.820700e-09,1.111300e-11,1.196500e-12,3.782900e-08,9.415900e-09,MBA,0.43301
1,a0000002,2000002,2 Pallas,2,Pallas,NaN,N,N,4.20,545.000,...,3.469400e-06,6.272400e-06,9.128200e-06,8.859100e-06,4.961300e-09,4.653600e-10,4.078700e-05,3.680700e-06,MBA,0.35936
2,a0000003,2000003,3 Juno,3,Juno,NaN,N,N,5.33,246.596,...,3.223100e-06,1.664600e-05,1.772100e-05,8.110400e-06,4.363900e-09,4.413400e-10,3.528800e-05,3.107200e-06,MBA,0.33848
3,a0000004,2000004,4 Vesta,4,Vesta,NaN,N,N,3.00,525.400,...,2.170600e-07,3.880800e-07,1.789300e-07,1.206800e-06,1.648600e-09,2.612500e-10,4.103700e-06,1.274900e-06,MBA,0.39980
4,a0000005,2000005,5 Astraea,5,Astraea,NaN,N,N,6.90,106.699,...,2.740800e-06,2.894900e-05,2.984200e-05,8.303800e-06,4.729000e-09,5.522700e-10,3.474300e-05,3.490500e-06,MBA,0.52191


In [88]:
df5 = df5.drop(columns=['spkid', 'full_name', 'pdes', 'name', 'prefix', 'albedo', 'diameter_sigma', 'orbit_id', 'epoch',
       'epoch_mjd', 'epoch_cal', 'equinox', 'e', 'a', 'q', 'i', 'om', 'w',
       'ma', 'ad', 'n', 'tp', 'tp_cal', 'per', 'per_y', 'moid_ld',
       'sigma_e', 'sigma_a', 'sigma_q', 'sigma_i', 'sigma_om', 'sigma_w',
       'sigma_ma', 'sigma_ad', 'sigma_n', 'sigma_tp', 'sigma_per'])

df5 = df5.set_index("id")

df5.head()

,neo,pha,H,diameter,moid,class,rms
id,,,,,,,
a0000001,N,N,3.40,939.400,1.59478,MBA,0.43301
a0000002,N,N,4.20,545.000,1.23429,MBA,0.35936
a0000003,N,N,5.33,246.596,1.03429,MBA,0.33848
a0000004,N,N,3.00,525.400,1.13948,MBA,0.39980
a0000005,N,N,6.90,106.699,1.09575,MBA,0.52191


In [89]:
df5 = df5.dropna()

In [90]:
#There are only a few 1000 more cases so this may not impact accuracy much
len(df5.index)

132045

In [91]:
#Encode neo and pha N and Y values as 0s and 1s

# Call the encode_function on the neo and pha columns
df5["neo"] = df5["neo"].apply(encode)
df5["pha"] = df5["pha"].apply(encode)

#Review the df
df5.head()

,neo,pha,H,diameter,moid,class,rms
id,,,,,,,
a0000001,0,0,3.40,939.400,1.59478,MBA,0.43301
a0000002,0,0,4.20,545.000,1.23429,MBA,0.35936
a0000003,0,0,5.33,246.596,1.03429,MBA,0.33848
a0000004,0,0,3.00,525.400,1.13948,MBA,0.39980
a0000005,0,0,6.90,106.699,1.09575,MBA,0.52191


In [92]:
#Encode class values as numbers

# Call the encode_function on the neo and pha columns
df5["class"] = df5["class"].apply(encode2)

#Review the df
df5.head()

,neo,pha,H,diameter,moid,class,rms
id,,,,,,,
a0000001,0,0,3.40,939.400,1.59478,0,0.43301
a0000002,0,0,4.20,545.000,1.23429,0,0.35936
a0000003,0,0,5.33,246.596,1.03429,0,0.33848
a0000004,0,0,3.00,525.400,1.13948,0,0.39980
a0000005,0,0,6.90,106.699,1.09575,0,0.52191


In [93]:
# Remove target (pha) from features data
y = df5.pha.values
X = df5.drop(columns="pha").values


# Split training/test datasets
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42, stratify=y)

In [94]:
# Preprocess numerical data for neural network

# Create a StandardScaler instances
scaler = StandardScaler()

# Fit the StandardScaler
X_scaler = scaler.fit(X_train)

# Scale the data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

In [95]:
# Define the deep learning model 
nn_model = tf.keras.models.Sequential()
nn_model.add(tf.keras.layers.Dense(units=6, activation="relu", input_dim=6))
nn_model.add(tf.keras.layers.Dense(units=3, activation="relu"))
nn_model.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))

# Compile the Sequential model together and customize metrics
nn_model.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

# Train the model
fit_model = nn_model.fit(X_train, y_train, epochs=10)

# Evaluate the model using the test data
model_loss, model_accuracy = nn_model.evaluate(X_test,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

Epoch 1/10
3095/3095 [==============================] - 24s 7ms/step - loss: 0.0429 - accuracy: 0.9983
Epoch 2/10
3095/3095 [==============================] - 23s 7ms/step - loss: 0.0033 - accuracy: 0.9983
Epoch 3/10
3095/3095 [==============================] - 22s 7ms/step - loss: 0.0032 - accuracy: 0.9983
Epoch 4/10
3095/3095 [==============================] - 22s 7ms/step - loss: 0.0031 - accuracy: 0.9983
Epoch 5/10
3095/3095 [==============================] - 22s 7ms/step - loss: 0.0030 - accuracy: 0.9983
Epoch 6/10
3095/3095 [==============================] - 22s 7ms/step - loss: 0.0029 - accuracy: 0.9983
Epoch 7/10
3095/3095 [==============================] - 23s 7ms/step - loss: 0.0028 - accuracy: 0.9985
Epoch 8/10
3095/3095 [==============================] - 22s 7ms/step - loss: 0.0026 - accuracy: 0.9987
Epoch 9/10
3095/3095 [==============================] - 23s 7ms/step - loss: 0.0026 - accuracy: 0.9987
Epoch 10/10
3095/3095 [==============================] - 23s 7ms/step - l

Model Accuracy is back up to 99.9% with the 6 features from df3 and slightly more data for training.